In [3]:
import pandas as pd

df = pd.read_csv("../data/merged_data_clean.csv")
df.head()


,order_id,order_date,customer_segment,priority,product_category,order_value_inr,origin,destination,special_handling,carrier,...,rating,feedback_text,would_recommend,issue_category,total_cost,delivery_delay_days,is_delayed,cost_per_km,traffic_delay_hours,priority_encoded
0,ORD000001,2025-10-09,Individual,Express,Industrial,238.73,Kolkata,Hyderabad,Unknown,SpeedyLogistics,...,4.0,Unknown,Unknown,Unknown,458.45,1.0,1,3.004456,0.350000,3
1,ORD000002,2025-09-29,SMB,Express,Industrial,17.01,Hyderabad,Kolkata,Unknown,SpeedyLogistics,...,1.0,"Great service, very fast delivery!",Yes,Timing,492.79,1.0,1,1.361110,0.550000,3
2,ORD000003,2025-09-15,SMB,Economy,Industrial,3024.95,Mumbai,Pune,Unknown,SpeedyLogistics,...,3.0,"Perfect condition, thank you",No,Timing,1078.93,5.0,1,2.075903,0.033333,1
3,ORD000004,2025-10-13,Individual,Economy,Fashion,56.74,Hyderabad,Ahmedabad,Unknown,QuickShip,...,4.0,Unknown,Unknown,Unknown,651.30,0.0,0,1.204171,1.866667,1
4,ORD000005,2025-09-08,SMB,Standard,Fashion,19148.65,Chennai,Mumbai,Unknown,SpeedyLogistics,...,4.0,Unknown,Unknown,Unknown,625.24,1.0,1,0.499569,0.166667,2


In [4]:
df.info()
df.describe().T.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 40 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   order_id                 200 non-null    object 
 1   order_date               200 non-null    object 
 2   customer_segment         200 non-null    object 
 3   priority                 200 non-null    object 
 4   product_category         200 non-null    object 
 5   order_value_inr          200 non-null    float64
 6   origin                   200 non-null    object 
 7   destination              200 non-null    object 
 8   special_handling         200 non-null    object 
 9   carrier                  200 non-null    object 
 10  promised_delivery_days   200 non-null    float64
 11  actual_delivery_days     200 non-null    float64
 12  delivery_status          200 non-null    object 
 13  quality_issue            200 non-null    object 
 14  customer_rating          2

,count,mean,std,min,25%,50%,75%,max
order_value_inr,200.0,1850.4030,4592.326209,1.91,150.04,500.855,1371.2825,47177.07
promised_delivery_days,200.0,4.3250,2.283736,1.00,3.00,4.000,5.0000,10.00
actual_delivery_days,200.0,5.3850,2.804658,1.00,3.75,5.000,6.0000,15.00
customer_rating,200.0,3.7300,1.167870,1.00,3.00,4.000,5.0000,5.00
delivery_cost_inr,200.0,574.5245,180.842333,150.84,460.42,575.580,639.6250,1178.01


In [5]:
cat_cols = df.select_dtypes(include=["object"]).columns
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)
df.head()


,order_value_inr,promised_delivery_days,actual_delivery_days,customer_rating,delivery_cost_inr,distance_km,fuel_consumption_l,toll_charges_inr,traffic_delay_minutes,fuel_cost,...,"feedback_text_Perfect condition, thank you",feedback_text_Poor customer service response,feedback_text_Unknown,feedback_text_Wrong item delivered,would_recommend_Unknown,would_recommend_Yes,issue_category_Quality,issue_category_Service,issue_category_Timing,issue_category_Unknown
0,238.73,1.0,2.0,3.0,387.86,152.59,23.02,122.08,21.0,151.36,...,False,False,True,False,True,False,False,False,False,True
1,17.01,2.0,3.0,1.0,430.19,362.05,43.98,289.64,33.0,157.32,...,False,False,False,False,False,True,False,False,True,False
2,3024.95,10.0,15.0,3.0,1039.19,519.74,65.75,415.79,2.0,322.61,...,True,False,False,False,False,False,False,False,True,False
3,56.74,5.0,5.0,5.0,599.01,540.87,61.85,432.70,112.0,215.33,...,False,False,True,False,True,False,False,False,False,True
4,19148.65,4.0,5.0,3.0,537.90,1251.56,147.54,1001.25,10.0,191.92,...,False,False,True,False,True,False,False,False,False,True


In [6]:
X = df.drop("is_delayed", axis=1)
y = df["is_delayed"]


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [8]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=200, 
    max_depth=10, 
    random_state=42
)
model.fit(X_train, y_train)


RandomForestClassifier(max_depth=10, n_estimators=200, random_state=42)

In [9]:
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:,1]

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nROC-AUC:", roc_auc_score(y_test, y_proba))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 1.0

ROC-AUC: 1.0

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        24

    accuracy                           1.00        40
   macro avg       1.00      1.00      1.00        40
weighted avg       1.00      1.00      1.00        40



In [10]:
df["predicted_delay_probability"] = model.predict_proba(X)[:,1]
df["predicted_is_delayed"] = model.predict(X)
df.head()


,order_value_inr,promised_delivery_days,actual_delivery_days,customer_rating,delivery_cost_inr,distance_km,fuel_consumption_l,toll_charges_inr,traffic_delay_minutes,fuel_cost,...,feedback_text_Unknown,feedback_text_Wrong item delivered,would_recommend_Unknown,would_recommend_Yes,issue_category_Quality,issue_category_Service,issue_category_Timing,issue_category_Unknown,predicted_delay_probability,predicted_is_delayed
0,238.73,1.0,2.0,3.0,387.86,152.59,23.02,122.08,21.0,151.36,...,True,False,True,False,False,False,False,True,0.773754,1
1,17.01,2.0,3.0,1.0,430.19,362.05,43.98,289.64,33.0,157.32,...,False,False,False,True,False,False,True,False,0.889278,1
2,3024.95,10.0,15.0,3.0,1039.19,519.74,65.75,415.79,2.0,322.61,...,False,False,False,False,False,False,True,False,0.910380,1
3,56.74,5.0,5.0,5.0,599.01,540.87,61.85,432.70,112.0,215.33,...,True,False,True,False,False,False,False,True,0.052200,0
4,19148.65,4.0,5.0,3.0,537.90,1251.56,147.54,1001.25,10.0,191.92,...,True,False,True,False,False,False,False,True,0.861748,1


In [11]:
import joblib

joblib.dump(model, "../app/model.pkl")
df.to_csv("../data/final_predictions.csv", index=False)
